In [39]:
import numpy
from PIL import Image 
import numpy as np
from skimage.io import imsave
import os
import glob

In [40]:
# Properties of Partially Homomorphic Encryption
def lcm(a, b):
    """
    lcm(a, b)
    returns Lowest Common Multiple of a and b
    """
    return a * b // xgcd(a,b)[0]

def xgcd(a, b):
    """
    xgcd (a, b)
    returns (g, x, y) according to the Extended Euclidean Algorithm
    such that, ax + by = g
    """
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = xgcd(b % a, a)
        return (g, x - (b // a) * y, y)

def multiplicative_inverse(a, modulus):
    """
    multiplicative_inverse(a, modulus)
    returns x: multiplicative inverse of a
    such that, a * x = 1 (mod modulus)
    """
    g, x, y = xgcd(a, modulus)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % modulus


def binary_exponent(base, exponent, modulus):
    """
    modular_binary_exponent( base, exponent, modulus)
    
    args:
        base
        exponent
        modulus
    
    generates:
        base ^ exponent (mod modulus)
    
        along with intermediate results from binary exponentiation
        required for Rabin-Miller primality test
    """
    if modulus == 1:
        yield 0
        return
    bitmask = 1 << exponent.bit_length() - 1
    res = 1
    while bitmask:
        res = (res * res) % modulus
        if bitmask & exponent:
            res = (res * base) % modulus
        yield res
        bitmask >>= 1

In [41]:
# Properties of Partially Homomorphic Encryption
import random
#import ModularArithmetic as mod

def is_probably_prime(n):
    """
    is_probably_prime(n)
    args:
        n
    returns:
        Boolean
    returns True if n is a probable prime
    based on the Rabin-Miller primality test
    """
    tests = max(64, n.bit_length())
    for i in range(tests):
        rand = random.randint(1,n-1)
        return 1 in binary_exponent(rand, n-1, n)

def generate_prime(bitlen=128):
    """
    generate_prime(bitlen)
    args:
        bitlen: length of number to be generated (default: 128)
    returns:
        a probable prime of bitlen bits
        based on the Rabin-Miller primality test
    """
    n = random.getrandbits(bitlen) | 1<<(bitlen-1) | 1
    while not is_probably_prime(n):
        n = random.getrandbits(bitlen) | 1<<(bitlen-1) | 1
    return n

In [42]:
# Properties of Partially Homomorphic Encryption
import random

#import ModularArithmetic
#import RabinMiller

class PrivateKey:
    """
    PrivateKey object contains λ and μ
    in accordance to the Paillier Cryptosystem
    
    args:
        p: a prime number
        q: another prime number
        (p and q are of equal length)
        n: product of p and q
        
    attributes:
        λ: lowest common multiple of p-1 and q-1
        ∵ p and q are of equal length we can use the simplification,
        μ: modular multiplicative inverse of λ and n
    """
    
    def __init__(self, p, q, n):

        self.λ = lcm( p-1, q-1)
        self.μ = multiplicative_inverse( self.λ, n)
        
    def __repr__(self):
        return ("---\nPrivate Key :\nλ:\t"+str(self.λ) +"\nμ:\t"+str(self.μ) +"\n---")


class PublicKey:
    """
    Public Key object contains n and g
    in accordance to the Paillier Cryptosystem
    
    args:
        n: product of two equal lenght prime numbers
    
    attributes:
        n: product of two primes
        g: a random number such that,
        multiplicative order of g in n^2 is a multiple of n
        
        ∵ p and q are of equal length we can use a simplification of g = n+1
    """
    def __init__(self, n):
        self.n = n
        self.nsq = n * n
        self.g = n+1
    
    def __repr__(self):
        return ("---\nPublic Key :\nn:\t"+ str(self.n) +"\n---")


def generate_keys(bitlen):
    """
    generate_keys( bitlen)
    
    args:
        bitlen: length of primes to be generated (default: 128)
    
    returns Public Private key pair as a tuple
    (PublicKey, PrivateKey)
    """
    
    p = generate_prime(bitlen)
    q = generate_prime(bitlen)
    n = p * q
    return (PublicKey(n), PrivateKey(p, q, n))

In [43]:
# Properties of Somewhat Homomorphic Encryption
import numpy
from PIL import Image
from PIL import Image
import numpy as np
# defining the big O constant
#bigOconstant = 2
#_lambda = 2
# Function to calculate eta value from the security parameter lambda
def eta(_lambda):			# eta is bit length of secret key 
	return (bigOconstant * _lambda * _lambda)
# Function to calculate gamma value from the security parameter lambda
def gamma(_lambda):
	return (bigOconstant * _lambda**5)
# Function to calculate tau value from the security parameter lambda
def tau(_lambda):			# tau is number of integers in public key
	return (gamma(_lambda) + _lambda)
# Function to calculate rho' value from security parameter lambda
def rho_dash(_lambda):
	return (2*_lambda)
# Function to generate a large value
# check_lsb = 1 if the number has to be odd and check msb = 1 to set the initial bit to be 1
def number_generator(num_of_bits, check_msb, check_lsb):
	number_count = num_of_bits / 32
	number_count = int(number_count)
	base = 2**32
	rand_num ={}
	if (number_count == 0):
		x = int(numpy.random.uniform(0,2**num_of_bits))
	else :
		for i in range(number_count):
			rand_num[i] = int(numpy.random.uniform(0,2**32))
		if (check_lsb == 1):
			rand_num[0] = ((rand_num[0] & ~1 ) + 1)
		if (check_msb == 1) :
			rand_num[number_count-1] = ((rand_num[number_count-1] & ~2**31 ) + 2**31)
		x=0
	# number = x0*base**0 + x1*base**1 + x2*base**2 + x3*base**3 + ...
		for i in range(number_count):
			x +=  rand_num[i] * base**i
	return x

# Function to calculate the secret key which is an odd eta-bit integer
def secret_key(_lambda):
	eta_value = eta(_lambda)
	secret_key = number_generator(eta_value, 1, 1)
	return secret_key	
# Function to find the different values of public key
def pubkey_distribution(_lambda, secret_key):
	#q_bound = int(math.ceil(math.log((2**gamma(_lambda) / secret_key),2)))
	#q = number_generator(q_bound, 0)
	q = int(numpy.random.uniform(0,2**256))
	r = (1 - 2*(numpy.random.randint(0,2))) * number_generator(_lambda, 0, 0)
	x = secret_key * q + r
	return x
	
# Function to calculate the public key list
def public_keyS(_lambda, secret_key):
	while True:
		pubkey = [pubkey_distribution(_lambda, secret_key) for i in range(tau(_lambda))]
		pubkey_max = max(pubkey)	# get largest value from list	
		pubkey_mod_sk = pubkey_max % secret_key
		# loop till largest value is odd and the value modulus secret key is even
		if (pubkey_max & 1) and (pubkey_mod_sk % 2 == 0):
			break
	return pubkey

# Function to generate a random subset S from {1, 2, 3, .... tau}
def generate_random_subset(_tau):
	set_size = numpy.random.randint(1, _tau)
	S = set()
	while len(S) < set_size:
		x = numpy.random.randint(1, _tau)
		S.add(x)
	return S
# Properties of Somewhat Homomorphic Encryption
# Function to encrypt one value(Hybrid Encryption function combines PHE and SHE)	
def Hybrid_encryption(_lambda, public_keyS, plaintext, public_key):
    """
		Hybrid_encryption(_lambda, public_keyS, plaintext, public_key)
			args:
				_lambda : SHE security parameter lambda
				public_keyS: SHE public_keyS function 
				plaintext : number to be encrypted
				public_key : Paillier Publickey object
			returns:
				c: encryption of plaintext
				such that c = (plaintext + 2*r + 2*sum_x) (mod n ^ 2)
				where, r(PHE scheme) is a random number in n(PHE scheme) such that r and n are coprime and 
				sum_x(SHE scheme) is the summation of xi's for i belongs to S i.e subset of {1,2,3 ... tau}
    """
    r = random.randint( 1, public_key.n-1)
    while not xgcd( r, public_key.n)[0] == 1:
        r = random.randint( 1, public_key.n)   
    random_subset = generate_random_subset(tau(_lambda))
    sum_x = 0
	# the summation of xi's for i belongs to S i.e subset of {1,2,3 ... tau}
    for i in random_subset:
        sum_x += public_keyS[i]
    c = (plaintext + 2*r + 2*sum_x) % public_key.nsq 
    return c

In [44]:
# Function call to encrypt image
def ImgEncrypt(_lambda, pk, plainimg, pub):
    """
	ImgEncrypt(_lambda, pk, plainimg, pub)
		args:
			_lambda : SHE security parameter lambda
			pk : SHE public_keyS function 
			plainimg : PIL Image object
			pub : Paillier Publickey object
		returns:
			cipherimg: Encryption of plainimg
    """
    cipherimg = np.asarray(plainimg)
    shape = cipherimg.shape
    cipherimg = cipherimg.flatten().tolist()
    cipherimg = [Hybrid_encryption(_lambda, pk, pix, pub) for pix in cipherimg]
    
    return np.asarray(cipherimg).reshape(shape)

In [ ]:
import time
start = time.time()
bigOconstant = 2
_lambda = 2
# function call to generate secret key(Properties of Somewhat Homomorphic Encryption)
sk = secret_key(_lambda)
# function call to generate public key list(Properties of Somewhat Homomorphic Encryption)		
pk = public_keyS(_lambda, sk)
# function call to generate tuple key Public Private key (Properties of Partially Homomorphic Encryption)		
pub, priv = generate_keys(bitlen=128) 
path = '1' # path of dataset file
for filename in glob.glob(os.path.join(path, '*.jpg')):
    with open(os.path.join(os.getcwd(), filename), 'r+') as f:
        print(filename)
        a1 = filename
        plainimg = Image.open(a1)
        cipherimg = ImgEncrypt(_lambda, pk, plainimg, pub)
        arra = np.array(cipherimg, dtype = 'float')
        imsave(a1,arra)